In [1]:
# first run installations and data download
# ! pip install -U llama-index llama-index-vector-stores-milvus pymilvus llama-index-llms-openai llama-index-readers-file
# ! mkdir -p './data/10k/'
# ! curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
# ! curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'
# ! mv 'lyft_2021.pdf' 'uber_2021.pdf' './data/10k/'

In [9]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from pymilvus import connections
from llama_index.vector_stores.milvus import MilvusVectorStore

In [10]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [11]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    vector_store_lyft = MilvusVectorStore(dim=1536, collection_name="lyft", overwrite=True)
    vector_store_uber = MilvusVectorStore(dim=1536, collection_name="uber", overwrite=True)
    storage_context_lyft = StorageContext.from_defaults(vector_store=vector_store_lyft)
    storage_context_uber = StorageContext.from_defaults(vector_store=vector_store_uber)
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, storage_context=storage_context_lyft)
    uber_index = VectorStoreIndex.from_documents(uber_docs, storage_context=storage_context_uber)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

RPC error: [__internal_register], <MilvusException: (code=65535, message=empty grpc client: find no available rootcoord, check rootcoord state)>, <Time:{'RPC start': '2024-03-02 10:30:15.703147', 'RPC error': '2024-03-02 10:31:08.328612'}>
Failed to create new connection using: cb91a5ef6aef4fc88ab7b2851260d393


MilvusException: <MilvusException: (code=65535, message=empty grpc client: find no available rootcoord, check rootcoord state)>

In [7]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [8]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [9]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [10]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [11]:
response = agent.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

Thought: I can use the "lyft_10k" tool to find information about Lyft's revenue growth in 2021.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue increased by 36% in 2021 compared to the prior year.
Thought: I have the information needed to answer the question.
Answer: Lyft's revenue growth in 2021 was 36%.
Lyft's revenue growth in 2021 was 36%.
